# FinnGenn Virus Both Directions

In [1]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
import warnings
warnings.filterwarnings("ignore")

#Directory in Biowulf
os.chdir('/PATH/TO/FinnGen_Data')

In [2]:
#Loading all the disease files
#Files downloaded on 5-5-22
AD_finn = pd.read_csv('G6_ALZHEIMER_survival-analyses.csv')
ALS_finn = pd.read_csv('G6_ALS_survival-analyses.csv')
dementia_finn = pd.read_csv('F5_DEMENTIA_survival-analyses.csv')
MS_finn = pd.read_csv('G6_MS_survival-analyses.csv')
PD_finn = pd.read_csv('G6_PARKINSON_survival-analyses.csv')
vascular_finn = pd.read_csv('F5_VASCDEM_survival-analyses.csv')
#FTD_finn = pd.read_csv('FTD_survival-analyses.csv') #no people who had one of our viruses

#Loading our serach term, i.e. list of viruses
search_terms = pd.read_csv('/data/levineks/Virus/FinnGen_UKB_Codes.csv') 

In [3]:
#Pick your NDD here
NDD = AD_finn
ndd = "AD"
NDD_code = "G6_ALZHEIMER"

In [4]:
NDD

,prior_name,outcome_name,hr_lag,prior_longname,outcome_longname,hr,ci_min,ci_max,p,N
0,KRA_PSY_DEMENTIA_EXMORE,G6_ALZHEIMER,15,Any dementia (more control exclusions),Alzheimer disease,1.781949e+14,1.060687e+14,2.993666e+14,0.000000e+00,176
1,NEURODEGOTH,G6_ALZHEIMER,15,Other degenerative diseases of the nervous system,Alzheimer disease,2.105497e+11,9.592746e+10,4.621325e+11,0.000000e+00,33
2,NEURODEGOTH,G6_ALZHEIMER,5,Other degenerative diseases of the nervous system,Alzheimer disease,9.113661e+10,4.940394e+10,1.681218e+11,0.000000e+00,48
3,KRA_PSY_DEMENTIA_EXMORE,G6_ALZHEIMER,5,Any dementia (more control exclusions),Alzheimer disease,3.281573e+10,1.557496e+09,6.914126e+11,1.134089e-54,636
4,KRA_PSY_DEMENTIA_EXMORE,G6_ALZHEIMER,0,Any dementia (more control exclusions),Alzheimer disease,1.570308e+10,1.165493e+10,2.115728e+10,0.000000e+00,9404
...,...,...,...,...,...,...,...,...,...,...
3636,G6_ALZHEIMER,N14_FEMALEGENNONINF,15,Alzheimer disease,Noninflammatory disorders of female genital tract,6.042992e-02,3.258726e-02,1.120614e-01,5.269369e-19,11
3637,I9_HEARTFAIL,G6_ALZHEIMER,15,"Heart failure,strict",Alzheimer disease,5.609922e-02,1.115124e-02,2.822219e-01,4.744813e-04,742
3638,G6_ALZHEIMER,I9_REVASC,15,Alzheimer disease,Coronary revascularization (ANGIO or CABG),5.593670e-02,2.944925e-02,1.062477e-01,1.257535e-18,11
3639,G6_ALZHEIMER,AB1_INTESTINAL_INFECTIONS,1,Alzheimer disease,Intestinal infectious diseases,4.430876e-02,6.968915e-11,2.817176e+07,7.631477e-01,203


In [5]:
#Select rows that are in our list of possible viruses
virus_list = list(search_terms['phenocode'])
had_virus_before = NDD[NDD['prior_name'].isin(virus_list)]
had_virus_after = NDD[NDD['outcome_name'].isin(virus_list)]

In [6]:
#checking results
print(len(had_virus_before))
print(len(had_virus_after))

47
20


In [7]:
had_virus = pd.concat([had_virus_before, had_virus_after])
had_virus

,prior_name,outcome_name,hr_lag,prior_longname,outcome_longname,hr,ci_min,ci_max,p,N
96,AB1_VIRAL_ENCEPHALITIS_NOS,G6_ALZHEIMER,0,"Viral encephalitis, not elsewhere classified/u...",Alzheimer disease,30.717384,11.841534,79.682047,1.893280e-12,24
100,J10_INFLUENZA,G6_ALZHEIMER,1,All influenza,Alzheimer disease,30.294508,19.298979,47.554704,9.870332e-50,80
107,J10_VIRALPNEUMO,G6_ALZHEIMER,1,Viral pneumonia,Alzheimer disease,29.480530,14.839669,58.566106,4.381031e-22,26
153,J10_INFLUPNEU,G6_ALZHEIMER,1,Influenza and pneumonia,Alzheimer disease,22.995190,16.422046,32.199323,1.948180e-74,681
156,AB1_VIRAL_NOS,G6_ALZHEIMER,1,"Other viral diseases, not elsewhere classified",Alzheimer disease,22.620981,10.584606,48.344623,8.350117e-16,14
...,...,...,...,...,...,...,...,...,...,...
2459,G6_ALZHEIMER,J10_VIRALPNEUMO,15,Alzheimer disease,Viral pneumonia,1.968061,1.150082,3.367816,1.350481e-02,16
2725,G6_ALZHEIMER,J10_INFLUENZA,15,Alzheimer disease,All influenza,1.641326,1.061367,2.538189,2.589586e-02,25
2957,G6_ALZHEIMER,AB1_HERPES_SIMPLEX,0,Alzheimer disease,Herpesviral [herpes simplex] infections,1.388239,0.801416,2.404755,2.418988e-01,15
3070,G6_ALZHEIMER,INFLUENZA,15,Alzheimer disease,All influenza (not pneumonia),1.250460,0.716144,2.183430,4.318917e-01,14


In [8]:
#Adding FDR Correction

#Sort P-values
had_virus = had_virus.sort_values(by = "p")

#FDR Correction
rejected, p_corr = fdrcorrection(had_virus['p'], is_sorted=True)
had_virus['P_CORR'] = p_corr
had_virus['REJECTED'] = rejected

In [9]:
had_virus

,prior_name,outcome_name,hr_lag,prior_longname,outcome_longname,hr,ci_min,ci_max,p,N,P_CORR,REJECTED
594,G6_ALZHEIMER,J10_INFLUENZA,0,Alzheimer disease,All influenza,8.321309,6.722313,10.300648,2.318047e-84,194,1.553091e-82,True
153,J10_INFLUPNEU,G6_ALZHEIMER,1,Influenza and pneumonia,Alzheimer disease,22.995190,16.422046,32.199323,1.948180e-74,681,6.526402e-73,True
798,J10_INFLUPNEU,G6_ALZHEIMER,5,Influenza and pneumonia,Alzheimer disease,6.755561,5.384530,8.475691,3.282922e-61,662,7.331859e-60,True
659,G6_ALZHEIMER,J10_INFLUENZA,5,Alzheimer disease,All influenza,7.788497,6.023389,10.070856,3.092084e-55,108,5.179240e-54,True
100,J10_INFLUENZA,G6_ALZHEIMER,1,All influenza,Alzheimer disease,30.294508,19.298979,47.554704,9.870332e-50,80,1.322625e-48,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3157,J10_COLD,G6_ALZHEIMER,0,Acute nasopharyngitis(common cold),Alzheimer disease,1.154484,0.591120,2.254762,6.740301e-01,26,7.067970e-01,False
3351,AB1_VIRAL_HEMOR_FEVER_NOS,G6_ALZHEIMER,15,"Viral haemorrhagic fevers, not elsewhere class...",Alzheimer disease,0.842128,0.377041,1.880909,6.751494e-01,11,7.067970e-01,False
3205,AB1_VIRAL_OTHER_INTEST_INFECTIONS,G6_ALZHEIMER,15,Viral and other specified intestinal infections,Alzheimer disease,1.071487,0.429239,2.674696,8.823911e-01,40,9.095416e-01,False
3282,AB1_VIRAL_WARTS,G6_ALZHEIMER,0,Viral warts,Alzheimer disease,0.946108,0.231615,3.864696,9.384994e-01,39,9.527191e-01,False


In [10]:
#Making sure we have data in both directions
prior_list = had_virus['prior_name']
outcome_list = had_virus['outcome_name']

a_set = set(prior_list)
b_set = set(outcome_list)
if (a_set & b_set):
    both = (a_set & b_set)
else:
    print("No common elements")

both_lists = list(both)
both_lists.remove(NDD_code)
print(len(both_lists))
print(both_lists)

7
['J10_VIRALPNEUMO', 'G6_ENCEPHOTH', 'AB1_HERPES_SIMPLEX', 'AB1_VIRAL_OTHER_INTEST_INFECTIONS', 'J10_INFLUENZA', 'AB1_ZOSTER', 'AB1_VIRAL_NOS']


In [11]:
had_virus_before = had_virus[had_virus['prior_name'].isin(both_lists)]
had_virus_after = had_virus[had_virus['outcome_name'].isin(both_lists)]
had_virus = pd.concat([had_virus_before, had_virus_after])
print(len(had_virus))
had_virus

40


,prior_name,outcome_name,hr_lag,prior_longname,outcome_longname,hr,ci_min,ci_max,p,N,P_CORR,REJECTED
100,J10_INFLUENZA,G6_ALZHEIMER,1,All influenza,Alzheimer disease,30.294508,19.298979,47.554704,9.870332e-50,80,1.322625e-48,True
362,J10_INFLUENZA,G6_ALZHEIMER,5,All influenza,Alzheimer disease,12.612910,8.375495,18.994161,6.921081e-34,85,4.215567e-33,True
786,J10_INFLUENZA,G6_ALZHEIMER,0,All influenza,Alzheimer disease,6.809482,4.680332,9.907213,1.150327e-23,214,5.505138e-23,True
107,J10_VIRALPNEUMO,G6_ALZHEIMER,1,Viral pneumonia,Alzheimer disease,29.480530,14.839669,58.566106,4.381031e-22,26,1.834557e-21,True
156,AB1_VIRAL_NOS,G6_ALZHEIMER,1,"Other viral diseases, not elsewhere classified",Alzheimer disease,22.620981,10.584606,48.344623,8.350117e-16,14,3.108099e-15,True
337,AB1_ZOSTER,G6_ALZHEIMER,1,Zoster [herpes zoster],Alzheimer disease,13.109623,6.957304,24.702418,1.705863e-15,29,6.015410e-15,True
926,AB1_VIRAL_OTHER_INTEST_INFECTIONS,G6_ALZHEIMER,5,Viral and other specified intestinal infections,Alzheimer disease,5.990744,3.411089,10.521277,4.654494e-10,30,1.247404e-09,True
827,J10_VIRALPNEUMO,G6_ALZHEIMER,5,Viral pneumonia,Alzheimer disease,6.615443,3.485119,12.557413,7.553002e-09,25,1.946350e-08,True
379,AB1_VIRAL_OTHER_INTEST_INFECTIONS,G6_ALZHEIMER,1,Viral and other specified intestinal infections,Alzheimer disease,11.970398,5.086727,28.169476,1.305084e-08,30,3.238542e-08,True
1040,AB1_VIRAL_NOS,G6_ALZHEIMER,5,"Other viral diseases, not elsewhere classified",Alzheimer disease,5.335655,2.711041,10.501211,1.252999e-06,17,2.708094e-06,True


In [12]:
#Save to csv
had_virus.to_csv("/PATH/TO/regression_results/" + ndd + "_finn_gen_BOTH.csv", index=False)

In [13]:
#Load and check results
df2 = pd.read_csv("/PATH/TO/regression_results/" + ndd + "_finn_gen_BOTH.csv")
df2

,prior_name,outcome_name,hr_lag,prior_longname,outcome_longname,hr,ci_min,ci_max,p,N,P_CORR,REJECTED
0,J10_INFLUENZA,G6_ALZHEIMER,1,All influenza,Alzheimer disease,30.294508,19.298979,47.554704,9.870332e-50,80,1.322625e-48,True
1,J10_INFLUENZA,G6_ALZHEIMER,5,All influenza,Alzheimer disease,12.612910,8.375495,18.994161,6.921081e-34,85,4.215567e-33,True
2,J10_INFLUENZA,G6_ALZHEIMER,0,All influenza,Alzheimer disease,6.809482,4.680332,9.907213,1.150327e-23,214,5.505138e-23,True
3,J10_VIRALPNEUMO,G6_ALZHEIMER,1,Viral pneumonia,Alzheimer disease,29.480530,14.839669,58.566106,4.381031e-22,26,1.834557e-21,True
4,AB1_VIRAL_NOS,G6_ALZHEIMER,1,"Other viral diseases, not elsewhere classified",Alzheimer disease,22.620981,10.584606,48.344623,8.350117e-16,14,3.108099e-15,True
5,AB1_ZOSTER,G6_ALZHEIMER,1,Zoster [herpes zoster],Alzheimer disease,13.109623,6.957304,24.702418,1.705863e-15,29,6.015410e-15,True
6,AB1_VIRAL_OTHER_INTEST_INFECTIONS,G6_ALZHEIMER,5,Viral and other specified intestinal infections,Alzheimer disease,5.990744,3.411089,10.521277,4.654494e-10,30,1.247404e-09,True
7,J10_VIRALPNEUMO,G6_ALZHEIMER,5,Viral pneumonia,Alzheimer disease,6.615443,3.485119,12.557413,7.553002e-09,25,1.946350e-08,True
8,AB1_VIRAL_OTHER_INTEST_INFECTIONS,G6_ALZHEIMER,1,Viral and other specified intestinal infections,Alzheimer disease,11.970398,5.086727,28.169476,1.305084e-08,30,3.238542e-08,True
9,AB1_VIRAL_NOS,G6_ALZHEIMER,5,"Other viral diseases, not elsewhere classified",Alzheimer disease,5.335655,2.711041,10.501211,1.252999e-06,17,2.708094e-06,True
